In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
import bempp.api.shapes.fractals

In [2]:
h = 0.05
dist = 1.5


sphere1 = bempp.api.shapes.sphere(h=h, origin=((dist/2) + 1, 0, 0))
sphere2 = bempp.api.shapes.sphere(h=h, origin=(-((dist/2) + 1), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

for N_wn in [30, 40, 50, 60, 70, 100]:

    for ubound_wn in [8, 9, 10]:
        lbound_wn = 0
        quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)
        results = []
        for index2, point2 in enumerate(quad_points):

            wavenumber = -1j * np.log(point2)

            slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
            slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
            slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
            slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

            mat11 = slp11.weak_form().A
            mat12 = slp12.weak_form().A
            mat21 = slp21.weak_form().A
            mat22 = slp22.weak_form().A
            mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
            mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

            mat = np.block([[mat11,mat12],[mat21,mat22]])  
            mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
            mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

            inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                                  [mat21_zero, mat22_inv]])  

            combined_inverse_free = inv_free_mat2.dot(mat)
            sign, logdet = slogdet(combined_inverse_free)

            results.append(logdet)
        print(N_wn, np.trapz(-np.array(results) / quad_points, quad_points) / (2 * np.pi))

30 0.004677607732183618
30 0.004677610299632613
30 0.004677611218939785
40 0.004675288427113692
40 0.004675289846273479
40 0.004675290349533151
50 0.0046742369364865905
50 0.004674237835265853
50 0.004674238150950219
60 0.004673673033427651
60 0.004673673653229866
60 0.004673673868845562
70 0.004673335977107332
70 0.004673336430187327
70 0.004673336586305
100 0.004672864595744328
100 0.004672864815902405
100 0.0046728648894841665
